In [272]:
import math 
import numpy as np
import plotly.graph_objects as go
from utils import basic_dict, basic_dict_2, read_db

volume = 80 # Abschaetzung eines normal grossen Raumes


surface = {'wall1': 100, 'wall2': 20, 'wall3': 10, 'wall4': 10, 'wall5': 8, 'wall6': 8} # Abschaetzung eines normal grossen Raumes

alpha = {'125 Hz': [0.2, 0.1 ,0.2, 0.1, 0.1, .1, 0.1], 
        '250 Hz': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1], 
        '500 Hz': [0.02, 0.1, 0.2, 0.07, 0.1, 0.2, 0.1],
        '1 kHz': [0.1, 0.1, 0.08, 0.1, 0.1, 0.1, 0.1], 
        '2 kHz': [0.1, 0.04, 0.1, 0.2, 0.1, 0.1], 
        '4 kHz': [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
        } 

sub_surface = {'wall1':[5, 4], 'wall2':[6, 2], 'wall3': [7], 'wall4': [], 'wall5': [], 'wall6': [1, 2, 3]}

sub_alpha = {'125 Hz': {'wall1':[0.15, 0.36], 'wall2':[0.16, 0.12], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]},
            '250 Hz': {'wall1':[0.2, 0.1], 'wall2':[0.3, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]}, 
            '500 Hz': {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]}, 
            '1 kHz' : {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]},
            '2 kHz':  {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]},
            '4 kHz':  {'wall1':[0.2, 0.1], 'wall2':[0.2, 0.1], 'wall3': [0.13], 'wall4': [], 'wall5': [], 'wall6': [0.1, 0.2, 0.3]}  
            }       

peopleDescription = ['Person sitzend auf ungepolsterter Bestuhlung', 'Kind in Vorschuleinrichtungen']

numberOfPeople = [1, 2]

use = 'Musik'

class room: 
    '''Class inheriting all functions for calculations and making plots.'''

    def __init__(self, volume, surface, sub_surface, alpha, sub_alpha, peopleDescription, numberOfPeople, use):
        '''Function to initialize the class "room"'''
        self.input = {'Volume': volume, 'Surface': surface, 'Absorption coefficient': alpha}
        self.volume = volume
        self.surface = surface
        self.sub_surface = sub_surface
        self.alpha = alpha
        self.sub_alpha = sub_alpha
        self.use = use
        self.peopleDescription = peopleDescription
        self.numberOfPeople = numberOfPeople
        self.ErrorMessage = []

    def criticalDistance(self):
        '''
        Function to calculate the critical distance, where the energy densities of the direct and reflected soundfield are equal. 
        Calculation is made with an approximate formula based on statistical acoustics in a diffuse soundfield.
        '''
        
        criticalDistance = np.sqrt(self.equivalentAbsorptionSurface() / 50)

        return criticalDistance
        
    def equivalentAbsorptionSurface_walls(self):
        '''Function to calculate the equivalent absorption surface.'''

        equivalentAbsorptionSurface = basic_dict()

        wall_index = 0
        # equivalent absorption surface for main walls 
        for octaveBands in self.alpha:
            alphaList = self.alpha[octaveBands]

            for walls in self.surface:
                equivalentAbsorptionSurface[octaveBands] = equivalentAbsorptionSurface[octaveBands] + (self.surface[walls] - sum(self.sub_surface[walls])) * alphaList[wall_index]
                wall_index = wall_index + 1
            wall_index = 0

        # adding equivalent absorption surface for sub walls
        for octaveBands in self.sub_alpha.keys():
            sub_alphaDict = self.sub_alpha[octaveBands]
            for sub_walls in self.sub_surface.keys():
                sub_alphaList = sub_alphaDict[sub_walls]
                sub_surfaceList = self.sub_surface[sub_walls]
                    
                for sub_wall_index in range(len(sub_surfaceList)):
                    equivalentAbsorptionSurface[octaveBands] = equivalentAbsorptionSurface[octaveBands] + sub_surfaceList[sub_wall_index] * sub_alphaList[sub_wall_index]

        return equivalentAbsorptionSurface
    
    
    def equivalentAbsorptionSurface_people(self):
        '''
        Function to add equivalent absorption surface based on the number of people in the room and their specification regarding age and position (e.g. standing, sitting).
        Data retrieved from Table A.1 in DIN 18041
        '''

        equivalentAbsorptionSurface_people = basic_dict()

        people_db = read_db('equivalentAbsorptionSurface_people_data.csv')
        
        for index in range(len(peopleDescription)):
            equivalentAbsorptionSurface_people_list =  people_db[peopleDescription[index]]

            for index_list, octaveBands in enumerate(equivalentAbsorptionSurface_people):
                equivalentAbsorptionSurface_people[octaveBands] += numberOfPeople[index] * equivalentAbsorptionSurface_people_list[index_list]
            
        return equivalentAbsorptionSurface_people
    
    def equivalentAbsorptionSurface(self):
        equivalentAbsorptionSurface = basic_dict_2()

        for octaveBands in equivalentAbsorptionSurface:
            equivalentAbsorptionSurface[octaveBands] = self.equivalentAbsorptionSurface_walls()[octaveBands] + self.equivalentAbsorptionSurface_people()[octaveBands]

        return equivalentAbsorptionSurface
   
    def reverberationTime(self):
        '''Function to calculate the reverberation time.'''

        reverberationTimeSeconds = basic_dict()
        equivalentAbsorptionSurface = self.equivalentAbsorptionSurface()
        
        for octavebands in equivalentAbsorptionSurface:
            reverberationTimeSeconds[octavebands] = (self.volume / equivalentAbsorptionSurface[octavebands]) * 0.161

        return reverberationTimeSeconds
    
    def reverberationTime_ratio(self):
        '''Function to calculate the ratio of given reverberation time to wanted reverberation time. Wanted reverberation time is based on the rooms use case and its volume.'''

        reverberationTime_ratio = basic_dict()
        ReverberationTime_upperlimit = {'125 Hz':1.45 , '250 Hz':1.2 , '500 Hz':1.2 , '1 kHz':1.2, '2 kHz':1.2 , '4 kHz':1.2 }
        ReverberationTime_lowerlimit = {'125 Hz':0.65 , '250 Hz':0.8 , '500 Hz':0.8 , '1 kHz':0.8, '2 kHz':0.8 , '4 kHz':0.65 }

        # Calculation of the wanted reverberation time dependent on the use case given in DIN 18041 (and the volume in use case "Sport")
        if self.use == 'Musik':
            reverberationTime_wanted = 0.45 * math.log10(self.volume) + 0.07

        elif self.use == 'Sprache/Vortrag':
            reverberationTime_wanted = 0.37 * math.log10(self.volume) - 0.14

        elif self.use == 'Sprache/Vortrag inklusiv':    
            reverberationTime_wanted = 0.32 * math.log10(self.volume) - 0.17

        elif self.use == 'Unterricht/Kommunikation':
            reverberationTime_wanted = 0.32 * math.log10(self.volume) - 0.17

        elif self.use == 'Unterricht/Kommunikation inklusiv':
            reverberationTime_wanted = 0.26 * math.log10(self.volume) - 0.14

        elif self.use == 'Sport':
                if self.volume > 10000:        
                    reverberationTime_wanted = 2
                else:
                    reverberationTime_wanted = 0.75 * math.log10(self.volume) - 1

        # Calculation of the ratio of calculated reverberation time to wanted reverberation time from DIN 18041 
        for octaveBands in self.reverberationTime():
            reverberationTime_ratio[octaveBands] = self.reverberationTime()[octaveBands] / reverberationTime_wanted
            if reverberationTime_ratio[octaveBands] > ReverberationTime_upperlimit[octaveBands]:
                self.ErrorMessage.append(f'Nachhallzeit in Oktavband mit Mittenfrequenz {octaveBands} zu hoch')
            elif reverberationTime_ratio[octaveBands] < ReverberationTime_lowerlimit[octaveBands]:
                self.ErrorMessage.append(f'Nachhallzeit in Oktavband mit Mittenfrequenz {octaveBands} zu niedrig')

        return reverberationTime_ratio, self.ErrorMessage
    
    def plot_reverberationTime(self):
        '''Function, which returns a plot of the reverberation time in octave bands.'''

        freq = np.array([125,250,500,1000,2000,4000])
        reverberationTimeSeconds = self.reverberationTime() 

        fig = go.Figure()

        trace1 = go.Bar(x = freq, 
                        y = list(reverberationTimeSeconds.values()), 
                        name = 'bar', 
                        marker_color = 'blue', 
                        showlegend= True, 
                        width=.2)
        
        fig.add_trace(trace1)

        fig.update_layout(xaxis_title = 'Frequenz [Hz]', 
                          yaxis_title = 'Nachhallzeit [s]', 
                          width = 1000, 
                          height = 600)
        
        fig.update_xaxes(type='category')

        return fig
    
    def plot_reverberationTime_ratio(self):
        '''
        Function, which returns a plot of the calculated reverberation time in comparison to the wanted reverberation time 
        and the allowed deviations in octave bands.'''
        
        frequencies = [125,250,500,1000,2000,4000]
        
        ReverberationTime_upperlimit = [1.45, 1.2, 1.2, 1.2, 1.2, 1.2]
        ReverberationTime_lowerlimit = [0.65, 0.8, 0.8, 0.8, 0.8, 0.65]

        reverberationTime_ratio = list(self.reverberationTime_ratio()[0].values())

        fig = go.Figure()

        trace1 = go.Scatter(x = frequencies, 
                            y = ReverberationTime_lowerlimit, 
                            name = 'Grenzen', 
                            marker_color = 'green', 
                            mode = 'lines', 
                            legendgroup = 'boundaries',
                            hoverinfo = 'skip'
                            )
        
        trace2 = go.Scatter(x = frequencies, 
                            y = ReverberationTime_upperlimit, 
                            marker_color = 'green', 
                            fill = 'tonexty', 
                            fillcolor = 'rgba(26, 199, 93, 0.1)',
                            mode = 'lines',
                            legendgroup = 'boundaries',
                            showlegend = False,
                            hoverinfo = 'skip',
                            )
        
        trace3 = go.Bar(x = frequencies, 
                        y = reverberationTime_ratio, 
                        name = 'Nachhallzeitenvergleich',
                        width = .2,
                        marker_color = 'rgba(28, 122, 255, 1)',
                        hovertemplate = 'Oktavband: %{x} Hz<br>Nachhallzeitenvergleich: %{y:.2f}<extra></extra>'
                        )
        
        fig.update_xaxes(type = 'category')       

        fig.update_layout(xaxis_title = 'Frequenz [Hz]',
                          yaxis_title = 'T / T_soll', 
                          width = 1000, 
                          height = 600, 
                        #   legend={'traceorder':'normal'},
                          hoverlabel = dict(bgcolor = 'rgba(28, 122, 255, .4)')
                          )
        
        fig.add_trace(trace1)
        fig.add_trace(trace2)
        fig.add_trace(trace3)
        
        return fig

room1 = room(volume , surface, sub_surface, alpha, sub_alpha, peopleDescription, numberOfPeople, use)
plot_reverberationTime = room1.plot_reverberationTime()
plot_reverberationTimeRatio = room1.plot_reverberationTime_ratio()

In [273]:
from fpdf import FPDF
from fpdf.enums import XPos, YPos
import json
import plotly.io as pio

class pdfprotocol(FPDF):

    def __init__(self, filename, plot_reverberationTime, plot_reverberationTimeRatio):
        # create object
        self.pdf = FPDF('P', 'mm', 'A4')
        self.filename = filename
        self.plot_reverberationTime = plot_reverberationTime
        self.plot_reverberationTimeRatio = plot_reverberationTimeRatio
        self.font = 'arial'

    def load_variables(self):
        json_file = open(self.filename)
        variables = json.load(json_file)
        json_file.close()

        return variables
    
    def header(self):
        self.pdf.set_font(self.font, 'B', 16)
        self.pdf.set_fill_color(211, 211, 211)
        self.pdf.cell(0, 10 ,'Raumakustik Protokoll', fill = True, new_x=XPos.LMARGIN, new_y=YPos.NEXT , align = 'C')
        self.pdf.ln(5)
        
    def footer(self):
        self.pdf.set_y(-15)
        self.pdf.set_font(self.font, 'I', 9)
        self.pdf.cell(0,10, f'Seite {self.pdf.page_no()}/{{nb}}', align = 'R')
        
    def basic_variables(self):
        use = self.load_variables()['usecase']
        volume = self.load_variables()['volume']
        number_walls = self.load_variables()['number_walls']
        persons = self.load_variables()['persons'] # True or False 7

        if persons == True: 
            number_people = self.load_variables()['number_people']
            return use, volume, number_walls, persons, number_people
        else: 
            return use, volume, number_walls, persons
    
    def wall_variables(self, index):
        '''
        Function to read variables of walls
        '''
        name = self.load_variables()['wall' + f'{index + 1}']['name']
        area = self.load_variables()['wall' + f'{index + 1}']['area']
        category = self.load_variables()['wall' + f'{index + 1}']['category']
        material = self.load_variables()['wall' + f'{index + 1}']['material']
        number_subareas = self.load_variables()['wall' + f'{index + 1}']['number_subareas']

        return name, area, category, material, number_subareas

    def subwall_variables(self, index, subindex):
        '''
        Function to read variables of subwalls
        '''
        area = self.load_variables()['wall' + f'{index + 1}']['subarea' + f'{subindex + 1}']['area']
        category = self.load_variables()['wall' + f'{index + 1}']['subarea' + f'{subindex + 1}']['category']
        material = self.load_variables()['wall' + f'{index + 1}']['subarea' + f'{subindex + 1}']['material']

        return area, category, material
    
    def people_variables(self, index):
        amount = self.load_variables()['person_type' + f'{index + 1}']['amount']
        type = self.load_variables()['person_type' + f'{index + 1}']['type']

        return amount, type
    
    def protocol(self):        
        # get total page number
        self.pdf.alias_nb_pages()
        # auto page break (margin: space from the bottom)
        self.pdf.set_auto_page_break(auto = True, margin = 15)
        # add page
        self.pdf.add_page()

        # pdf.set_text_color(220, 50 50) # rot
        
        # header
        self.header()
    
        # font of text
        self.pdf.set_font(self.font, '', 9)
        
        # usecase and volume
        # (width, height, 'text', ln = True/False -> cursor moves to the next line, border = True/False)
        self.pdf.cell(0, 5, f'Nutzungsart: {self.basic_variables()[0]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.pdf.cell(0, 5, f'Volumen: {self.basic_variables()[1]} m³', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.pdf.ln(3)
        
        # people
        if self.basic_variables()[3] == True:
            # header
            self.pdf.set_font(self.font, 'B', 11)
            self.pdf.cell(0, 5, f'Personen:', fill = True, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
            self.pdf.ln(1)
            self.pdf.set_font(self.font, '', 9)
            # people variables
            for index in range(self.basic_variables()[4]):
                # if there is more than one group of people write group 1, group 2... 
                if self.basic_variables()[4] != 1:
                    self.pdf.cell(0, 5, f'Personengruppe ' + f'{index + 1}:', new_x=XPos.LMARGIN, new_y=YPos.NEXT)

                self.pdf.cell(0, 5, f'Beschreibung: {self.people_variables(index)[1]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
                self.pdf.cell(0, 5, f'Anzahl: {self.people_variables(index)[0]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
                self.pdf.ln(1)

            self.pdf.ln(3)
        # main walls
        for index in range(self.basic_variables()[2]):
            self.pdf.set_font(self.font, 'B', 11)
            self.pdf.cell(0, 5, f'{self.wall_variables(index)[0]}', fill = True, new_x=XPos.LMARGIN, new_y=YPos.NEXT)
            self.pdf.ln(1)
            self.pdf.set_font(self.font, '', 9)
            

            self.pdf.cell(0, 5, f'Fläche: {self.wall_variables(index)[1]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
            self.pdf.cell(0, 5, f'Kategorie: {self.wall_variables(index)[2]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
            self.pdf.multi_cell(0, 5, f'Material: {self.wall_variables(index)[3]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
            self.pdf.ln(1)
            # sub walls
            for subindex in range(self.wall_variables(index)[4]):
                # if self.wall_variables(index)[4] > 1:
                self.pdf.cell(0, 5, f'Subwand {subindex + 1}:', new_x=XPos.LMARGIN, new_y=YPos.NEXT)

                self.pdf.cell(5, 5, '')
                self.pdf.cell(0, 5, f'Fläche: {self.subwall_variables(index, subindex)[0]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
                self.pdf.cell(5, 5, '')
                self.pdf.cell(0, 5, f'Kategorie: {self.subwall_variables(index, subindex)[1]}', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
                self.pdf.cell(5, 5, '')
                self.pdf.multi_cell(0, 5, f'Material: {self.subwall_variables(index, subindex)[2]}', 0,  new_x=XPos.LMARGIN, new_y=YPos.NEXT)
                self.pdf.ln(1)
            self.pdf.ln(2)

        # write plots
        pio.write_image(self.plot_reverberationTime, 'plot_reverberationTime', format = 'png')
        pio.write_image(self.plot_reverberationTimeRatio, 'plot_reverberationTimeRatio', format = 'png')

        # show plots
        self.pdf.add_page()
        self.pdf.ln(10)
        self.pdf.set_font(self.font, 'B', 14)
        self.pdf.cell(0, 5, 'Nachhallzeit', align = 'C', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.pdf.image('plot_reverberationTime', w = 180)
        self.pdf.ln(10)
        self.pdf.cell(0, 5, 'Nachhallzeitenvergleich', align = 'C', new_x=XPos.LMARGIN, new_y=YPos.NEXT)
        self.pdf.image('plot_reverberationTimeRatio', w = 180)

        # footer
        # self.footer()

        # output PDF file
        self.pdf.output('pdf_test.pdf')

In [274]:

pdf1 = pdfprotocol('save_test.json', plot_reverberationTime, plot_reverberationTimeRatio)

pdf1.protocol()